### Big confusion that  started this:
When I read from the text file (utf-8 encode) and do title2id, works fine! When I do the same thing from json (and again utf-8) encoded, shit happens! 

### tldr;
if you read title from normal text file, it's already binary and you can query, if you have decoded, you should encode again. 
If you read from json, you always should encode the title
You were lucky that in the `annonatate.py`, you used id and not title!

### A bit history

You encode a unicode string, meaning that, you convert it to byte representation

Unicode:
4 bytes, max 0x10FFFF
normally encoding, meaning that putting the actuall codepoint, is a waste of space

utf-8 is compact and blah blah!

Normally the files contain the encoded thing, and if what you do is merely byte operation, no need to decode and encode. The following piece of code works (meaning that the article is found, and the find is readable by vim) because the default string in python is ascii and not unicode, matches the file, and the database (which is also binary) (I am using ascii and binary interchangably, not quite sure though)




Experiment 1: I did this when I was a bit ignorant! The result should be obvious

This shows that if I don't touch anything, everythig is fine, Anarchism is an article with unicode text (alpha...), the second title is problamtic, and the third one is even worse (sometimes breaks when you want to title2id it). 

in Anarchism, there are multiple problomatic words that can be traced, such as the sentence with "verbal infinitive suffix -ίζειν, -izein" , surprisingly "people’s ‘mental enslavement’" is encoded strangely!

In [ ]:
import sys
sys.path.append('../../cgi-bin/')
from wikipedia import *

In [ ]:
r'<doc id="(.*)" url="(.*)" title="(.*)">'

import re
import json
query = "Anarchism"
query = "Asociación Alumni"
# query = "Atanasoff–Berry computer"
# query = "Bunge &amp; Born"

output1=open('out1', 'w')
output2=open('out2', 'w')
found = False
text=""
with open('/users/grad/sajadi/backup/wikipedia/20160305/texts/xmltexts/wiki_00') as f:
    for line in f:
        line=line.strip()
        r = re.match(r'<doc id="(.*)" url="(.*)" title="(.*)">',line)
        if r is not None:
            title=r.group(3)
            if title == query:
                found=True
                continue
        if not found:
            continue
        

        if re.match('</doc>', line):
            break
            
        text += line + '\n' 
output1.write(title+"\n"+text.strip())        
output1.close()
output2.write(json.dumps({"title":title, "text": text.strip()}, ensure_ascii=False))
output2.close()

wid=title2id(title.replace(" ","_"))
if wid is not None:
    print "Passed, id: ", wid
print 'done'

In the code I have written (tojson), I have  decoded/encoded. It made some part of the code complicated (encode before unquoting, and decode after). I could avoid that, skipping those lines and changing one line of the code:

`url=url.replace(u"\xA0"," ")`

by

`nbsp=hp.unescape("&nbsp;").encode('utf-8')
url=url.replace(nbsp," ")`

Buyt which one is better? Probably decoding/encoding. 

I wrote a code to see if it changes anything, obviously not when I think about it now, but then I was confused!

In [ ]:
r'<doc id="(.*)" url="(.*)" title="(.*)">'
from wikipedia import *
import re
import json
query = "Anarchism"
query = "Asociación Alumni".decode('utf-8')
query = "Atanasoff–Berry computer".decode('utf-8')
query = "Bunge &amp; Born".decode('utf-8')

output3=open('out3', 'w')
output4=open('out4', 'w')
found = False
text=""
with open('/users/grad/sajadi/backup/wikipedia/20160305/texts/xmltexts/wiki_00') as f:
    for line in f:
        line=line.decode('utf-8').strip()
        r = re.match(r'<doc id="(.*)" url="(.*)" title="(.*)">',line)
        if r is not None:
            title=r.group(3)
            if title == query:
                found=True
                continue
        if not found:
            continue
        

        if re.match('</doc>', line):
            break
            
        text += line + '\n' 
output3.write(json.dumps({"title":title, "text": text.strip()}, ensure_ascii=False).encode('utf-8'))
output3.close()
output4.write(json.dumps({"title":title.encode('utf-8'), "text": text.encode('utf-8').strip()}, ensure_ascii=False))
output4.close()

#wid=title2id(title.replace(" ","_"))
if wid is not None:
    print "Passed, id: ", wid
print 'done'

ok, so output 2 to 4 are the same, but how about 1 and 2?


In [ ]:
import re
with open('out1') as f:
    line1 = f.readline().strip()
    title1 = line1
    print title1
    
with open('out2') as f:
    line2 = f.readline().strip()
    
r = re.search(r'"title": "([^"]*)"', line2)
title2 = r.group(1)
print title2

print title1==title2    

title3 = json.loads(line2, encoding='utf-8')['title']
print title3
print title1 == title3

title4 = title3.encode('utf-8')
print title1 == title4


The above example kinda proves the data is totally the same in both of the files, and it's the json.loads that converts it to unicode by default, so I have to encode it if I want to send it to the database. Problem solved! 

** That encoding='utf-8' doens't do anything apparently! Neighter the decoding of the line. But I guess it's always better to decode.**

In [ ]:
with open('out2') as f:
    line1 = f.readline().strip()
line2=line1.decode('utf-8')
print line1==line2
print json.loads(line1) == json.loads(line2)



# How about Solr?? Should I send the binary? Does it save encoded or decoded? Yes, look down (basically don't worry about it)
## Experiment 1

In [ ]:
import requests
qstr = 'http://localhost:8983/solr/enwiki20160305/select'

query = ("Anarchism", "Asociación_Alumni", "Atanasoff–Berry_computer", "Bunge_&_Born")
wids = [title2id(q) for q in query]
titles = [id2title(wid) for wid in wids] 
for  title in titles:
    q='title:'+title
    params={'indent':'on', 'wt':'json',"rows":1, "fl":"id title", 'q':q}
    r = requests.get(qstr, params=params)
    r =  r.json()['response']['docs'][0]
    print r['title']

print r.js

## Open problem!!
Solrtagger, like java, uses utf-16, python utf-8. Can't get around it
Forexample, consider the first following sentence, T1, or the second, the opening text of an article, T2
The offset is shifter after the problematic character

In [6]:
import requests
T1=u'I ate 𩬅 in China and blah blah'

addr='http://localhost:8983/solr/enwiki20160305/select?indent=on&q=id:%22245791%22&wt=json'
r = requests.get(addr).json()['response']['docs']
T2=r[0]['opening_text']

text = T1
addr = 'http://localhost:8983/solr/enwikianchors20160305/tag'
params={'overlaps':'LONGEST_DOMINANT_RIGHT', 'tagsLimit':'5000', 'fl':'id','wt':'json','indent':'on', 'matchText':'true'}
#text=solr_escape(text)
r = requests.post(addr, params=params, data=text.encode('utf-8'))    

for tag in r.json()['tags']:
    #print  text[tag[1]:tag[3]], tag[5]
    assert text[tag[1]:tag[3]] == tag[5]


AssertionError: 